# Water Box Example -- GROMACS

In this simulation a GROMACS simulation of tip3p water is set up and run using `mBuild` and `GMSO`.

In [ ]:
import mbuild as mb

import gmso
from gmso import ForceField
from gmso.external import convert_mbuild
from gmso.formats.gro import write_gro
from gmso.formats.top import write_top

In [ ]:
%%writefile spce.mol2
@<TRIPOS>MOLECULE
RES
3 0 1 0 1
SMALL
NO_CHARGES
@<TRIPOS>CRYSIN
    3.0130     3.0130     3.0130    90.0000    90.0000    90.0000  1  1
@<TRIPOS>ATOM
       1 O            0.0000     0.0000     0.0000 O             1 RES     
       2 H           -0.6126    -0.7357     0.0000 H             1 RES     
       3 H           -0.5469     0.7762     0.0000 H             1 RES     
@<TRIPOS>BOND
     1     1     2    1
     2     1     3    1
@<TRIPOS>SUBSTRUCTURE
       1 RES             1 RESIDUE    0 **** ROOT      0

In [ ]:
# Load in single water (SPC/E) structure

water = mb.load("spce.mol2")
water = water.children[0]
water.name = "water"

# element_map = which site name corresponds to which atom_type name. 
# In the future atomtyping will be done through foyer. 
element_map = {"O": "opls_116",
               "H": "opls_117"}

In [ ]:
# Fill a box with 1000 water molecule

water_box = mb.fill_box(
    compound=water,     
    n_compounds=1000,
    density=1000,
)

In [ ]:
# Write out the SPC/E water xml file
%%writefile spce.xml
<?xml version='1.0' encoding='UTF-8'?>
<ForceField name="spce" version="0.0.1">
  <FFMetaData>
    <Units energy="kcal/mol" mass="amu" charge="elementary_charge" distance="nm"/>
  </FFMetaData>
  <AtomTypes expression="4 * epsilon * ((sigma/r)**12 - (sigma/r)**6)">
    <ParametersUnitDef parameter="epsilon" unit="kJ/mol"/>
    <ParametersUnitDef parameter="sigma" unit="nm"/>
    <AtomType name="opls_116" atomclass="OW" element="O" charge="-0.8476" mass="15.999" definition="O">
      <Parameters>
        <Parameter name="epsilon" value="0.650194"/>
        <Parameter name="sigma" value="0.316557"/>
      </Parameters>
    </AtomType>
    <AtomType name="opls_117" atomclass="HW" element="H" charge="0.4238" mass="1.00784" definition="H">
      <Parameters>
        <Parameter name="epsilon" value="0.0"/>
        <Parameter name="sigma" value="0.1"/>
      </Parameters>
    </AtomType>
  </AtomTypes>
  <BondTypes expression="0.5 * k * (r-r_eq)**2">
    <ParametersUnitDef parameter="k" unit="kJ/mol/nm**2"/>
    <ParametersUnitDef parameter="r_eq" unit="nm"/>
    <BondType name="BondType-Harmonic-1" type1="opls_116" type2="opls_117">
      <Parameters>
        <Parameter name="k" value="345000.0"/>
        <Parameter name="r_eq" value="0.1"/>
      </Parameters>
    </BondType>
  </BondTypes>
  <AngleTypes expression="0.5 * k * (theta - theta_eq)**2">
    <ParametersUnitDef parameter="k" unit="kJ/mol/degree**2"/>
    <ParametersUnitDef parameter="theta_eq" unit="degree"/>
    <AngleType name="AngleType-Harmonic-1" type1="opls_116" type2="opls_117" type3="opls_117">
      <Parameters>
        <Parameter name="k" value="383.0"/>
        <Parameter name="theta_eq" value="1.91061193"/>
      </Parameters>
    </AngleType>
  </AngleTypes>
</ForceField>

In [ ]:
# Load in topology forcefield

forcefield = ForceField("spce.xml")

In [ ]:
# Generate a topology from the mbuild compound

top = convert_mbuild.from_mbuild(water_box)

In [ ]:
# Assign atom types
for atom in top.sites:
    atom.atom_type = forcefield.atom_types[element_map[atom.name]]
    
# Assign bond types
for bond in top.bonds:
    bond.bond_type = bond.connection_type = forcefield.bond_types["opls_116~opls_117"]

# Create angles with correct atom type and add to top
for subtop in top.subtops:
    angle = gmso.core.angle.Angle(
        connection_members=[site for site in subtop.sites],
        name="opls_116~opls_117~opls_117",
        connection_type=forcefield.angle_types["opls_116~opls_117~opls_117"]
    )
    top.add_connection(angle, update_types=False)

top.update_topology()

In [ ]:
# Write out gro and top files

write_gro(top, "system.gro")
write_top(top, "system.top")

In [ ]:
# Lets look at the TOP file

!cat system.top

In [ ]:
# Write out an MDP file for an NPT simulation
%%writefile npt.mdp
constraints         = h-bonds
integrator          = md
nsteps              = 1000000
dt                  = 0.001

nstxtcout           = 10
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
rcoulomb            = 0.8
rvdw                = 0.8 

coulombtype         = PME

gen_vel             = yes

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1
ref_t               = 300

pcoupl              = Parrinello-Rahman
pcoupltype          = isotropic
tau-p               = 10
ref-p               = 1
compressibility     = 1e-5

In [ ]:
!gmx grompp -f npt.mdp -c system.gro -p system.top -o npt.tpr -maxwarn 2

In [ ]:
!gmx mdrun -v -s npt.tpr

In [ ]:
import mdtraj as md
traj = md.load("traj_comp.xtc", top="system.gro")

In [ ]:
traj